In [2]:
import rasterio as rio
from rasterio.warp import reproject, Resampling
from rasterio.transform import Affine


#### 1. 重采样

In [3]:
path_rsimg = 'data/section-6/s2_20m_6bands_chenggong.tif'
path_rsimg_resam = 'data/section-6/s2_20m_6bands_chenggong_resam.tif'


In [4]:
# --- 设置目标分辨率（可自定义X/Y方向不同）---
target_x_res = 30.0     # 目标X方向分辨率（米）
target_y_res = 40.0     # 目标Y方向分辨率（米）


In [5]:
## 读取原始栅格
with rio.open(path_rsimg) as src:
    data = src.read(1)       # 读取第一波段
    src_transform = src.transform  # 原始仿射变换矩阵
    src_crs = src.crs        # 原始坐标系
    src_width = src.width    # 原始宽度（像素）
    src_height = src.height  # 原始高度（像素）

print(src_transform)  # x方向分辨率，x方向偏移，左上角点x坐标；y方向...
# data.shape
print(src_crs)

| 20.00, 0.00, 268180.00|
| 0.00,-20.00, 2765440.00|
| 0.00, 0.00, 1.00|
EPSG:32648


In [6]:
# --- 1. 获取原始分辨率（X/Y方向）---
x_res = src_transform.a   # X方向分辨率
y_res = -src_transform.e  # Y方向分辨率, 负值表示北向

# --- 2. 计算重采样栅格参数 ---
# 2.1 缩放比例（宽度/高度）
x_scale = x_res / target_x_res  # X方向缩放比例
y_scale = y_res / target_y_res  # Y方向缩放比例

# 2.2 重采样栅格尺寸（整数像素）
dst_width = int(src_width * x_scale)
dst_height = int(src_height * y_scale)

# 2.3 重采样栅格仿射变换矩阵（调整分辨率，保持左上角原点不变）
dst_transform = Affine(
    a = target_x_res,     # X方向分辨率
    b = src_transform.b,  # X方向旋转（通常为0）
    c = src_transform.c,  # 左上角X坐标
    d = src_transform.d,  # Y方向旋转（通常为0）
    e = -target_y_res,    # Y方向分辨率（负值表示北向）
    f = src_transform.f   # 左上角Y坐标
    )


In [7]:
# --- 创建目标栅格并执行重采样 ---
with rio.open(
        path_rsimg_resam,
        "w",
        driver="GTiff",
        height=dst_height,
        width=dst_width,
        count=1,
        dtype=data.dtype,
        crs=src_crs,
        transform=dst_transform) as dst:

        reproject(
            source=data,  # 原数据
            src_transform=src_transform,  # 原数据仿射变换矩阵
            src_crs=src_crs,       # 原数据坐标系
            destination=rio.band(dst, 1),  # 目标数据（rio.band():波段写入）
            dst_transform=dst_transform,    # 目标数据仿射变换矩阵
            dst_crs=src_crs,    # 目标数据坐标系
            resampling=Resampling.bilinear,  # 可替换为其他方法
        )


#### 2. 重投影

In [8]:
import rasterio as rio
from rasterio.warp import reproject, Resampling, calculate_default_transform


In [9]:
from pyproj import CRS
path_rsimg = 'data/section-6/s2_20m_6bands_chenggong.tif'
path_rsimg_reproj = 'data/section-6/s2_20m_6bands_chenggong_reproj2wgs84.tif'
# target_crs = 'EPSG:4326'  ## 目标坐标系（WGS84）,rasterio支持不足
target_crs = CRS.from_epsg(4326)  ## 目标坐标系（WGS84）
target_crs = target_crs.to_wkt()   # WKT 格式, rasterio支持
target_crs

'GEOGCRS["WGS 84",ENSEMBLE["World Geodetic System 1984 ensemble",MEMBER["World Geodetic System 1984 (Transit)"],MEMBER["World Geodetic System 1984 (G730)"],MEMBER["World Geodetic System 1984 (G873)"],MEMBER["World Geodetic System 1984 (G1150)"],MEMBER["World Geodetic System 1984 (G1674)"],MEMBER["World Geodetic System 1984 (G1762)"],MEMBER["World Geodetic System 1984 (G2139)"],ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]],ENSEMBLEACCURACY[2.0]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]],AXIS["geodetic longitude (Lon)",east,ORDER[2],ANGLEUNIT["degree",0.0174532925199433]],USAGE[SCOPE["Horizontal component of 3D system."],AREA["World."],BBOX[-90,-180,90,180]],ID["EPSG",4326]]'

In [10]:
with rio.open(path_rsimg) as src:
    src_data = src.read()       # 读取所有波段
    src_crs = src.crs               # 原始坐标系
    src_count = src.count           # 波段数
    src_nodata = src.nodata         # 原始NoData值
    src_dtype = src.dtypes[0]       # 数据类型（假设所有波段一致）
    src_bounds = src.bounds         # 原始地理范围（left, bottom, right, top）


In [11]:
# 1. 计算重投影后影像参数（转换矩阵、宽度、高度）
dst_transform, dst_width, dst_height = calculate_default_transform(
    src_crs = src_crs,          # 原坐标系
    dst_crs = target_crs,       # 目标坐标系（WGS84）
    width = src.width,          # 原始宽度（像素）
    height = src.height,        # 原始高度（像素）
    left = src_bounds.left,
    bottom = src_bounds.bottom,
    right = src_bounds.right,
    top = src_bounds.top 
    )

dst_transform

Affine(0.00018901229645706938, 0.0, 102.70334409951235,
       0.0, -0.00018901229645706938, 24.990997116836738)

In [12]:
# 创建目标栅格文件
with rio.open(
        fp=path_rsimg_reproj,
        mode="w",
        driver="GTiff",
        height=dst_height,
        width=dst_width,
        count=src_count,  
        dtype=src_data.dtype,
        crs=target_crs,
        transform=dst_transform) as dst:
    # 执行重投影（循环，逐波段处理）
    for i in range(0, src_count):
        reproject(
            source=src_data[i],  # 输入数据
            src_transform=src_transform,
            src_crs=src_crs,
            src_nodata=src_nodata,
            destination=rio.band(dst, i+1),  # 目标波段, 注意索引从1开始
            dst_transform=dst_transform,
            dst_crs=target_crs,
            resampling=Resampling.bilinear,  # 重采样方法
        )


#### 影像裁剪

In [13]:
import rasterio as rio


In [14]:
# 输入输出路径
path_clip_1 = 'data/section-6/s2_20m_6bands_chenggong_clip1.tif'

# 定义裁剪范围（地理坐标：左、下、右、上）
clip_bounds_1 = (278726, 2748552, 282667, 2752483)  # 范围: (左, 下, 右, 上)
# 打开原始栅格
with rio.open(path_rsimg) as src:
    # 计算裁剪范围的像素窗口
    window = src.window(*clip_bounds_1)
    ## round_offsets():调整窗口的起始位置; round_shape():调整窗口的大小
    window = window.round_offsets().round_shape()  # 调整窗口，使其对齐到栅格数据 

    # 读取窗口内的数据, data为np.array数组
    data = src.read(window=window)
    # 计算裁剪后的新变换矩阵
    new_transform = src.window_transform(window)

    # 写入裁剪结果
    with rio.open(
            fp=path_clip_1,
            mode="w",
            driver="GTiff",
            height=window.height,
            width=window.width,
            count=src.count,
            dtype=data.dtype,
            crs=src.crs,
            transform=new_transform) as dst:
        dst.write(data)


c:\Users\HP\miniconda3\envs\gis-course\lib\site-packages\rasterio\windows.py:729: RasterioDeprecationWarning: round_shape is deprecated and will be removed in Rasterio 2.0.0.
  warnings.warn(


In [ ]:
## 基于矢量裁剪
import geopandas as gpd
from rasterio.mask import mask
path_rsimg = 'data/section-6/s2_20m_6bands_chenggong.tif'
path_ynu_vec = 'data/section-6/ynu.gpkg'
path_rsimg_clip_2 = 'data/section-6/s2_20m_6bands_chenggong_clip_2.tif'


In [ ]:
polygons = gpd.read_file(path_ynu_vec).to_crs(epsg=32648).geometry.values

with rio.open(path_rsimg) as src:
    # 执行掩膜裁剪
    crop_data, crop_transform = mask(
        src,
        polygons,
        crop=True,           # 裁剪到最小外接矩形
        all_touched=False,   # 只包含完全在内部的像素
        nodata=src.nodata    # 使用原始NoData值
    )

    # 更新元数据
    meta_ = src.meta.copy()
    meta_.update({
        'height': crop_data.shape[1],
        'width': crop_data.shape[2],
        'transform': crop_transform
    })

    with rio.open(path_rsimg_clip_2, 'w', **meta_) as dst:
        dst.write(crop_data)


#### 影像拼接

In [ ]:
import rasterio
from rasterio.merge import merge

# 输入文件列表
path_rsimg_1 = 'data/data-section-6/s2_20m_6bands_chenggong_clip1.tif'
path_rsimg_2 = 'data/data-section-6/s2_20m_6bands_chenggong_clip2.tif'
path_mosaic = 'data/data-section-6/s2_20m_6bands_chenggong_mosaic.tif'


In [16]:
# 读取所有栅格
src_1 = rio.open(path_rsimg_1)
src_2 = rio.open(path_rsimg_2)

# 执行拼接
merged_data, merged_transform = merge([src_1, src_2])

# 更新元数据
meta = src_1.meta.copy()
meta.update({
    "height": merged_data.shape[1],
    "width": merged_data.shape[2],
    "transform": merged_transform
    })
meta


{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 547,
 'height': 531,
 'count': 4,
 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 48N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
 'transform': Affine(10.0, 0.0, 278720.0,
        0.0, -10.0, 2752490.0)}

In [ ]:
## 写入结果
with rasterio.open(fp=path_mosaic, 
                mode="w", 
                driver="GTiff",
                height=meta['height'],
                width=meta['width'],
                count=meta['count'],
                dtype=meta['dtype'],
                crs=meta['crs'],
                transform=meta['transform']) as dst:
    dst.write(merged_data)

# 关闭所有输入文件
src_1.close()
src_2.close()

